In [ ]:
import pygame
import psycopg2
import pandas as pd
from pygame.locals import *
import config

# PostgreSQL connection setup
conn = psycopg2.connect(
    dbname="PhoneBook",
    user=config.DB_USER,
    password=config.DB_PASSWORD,
    host=config.DB_HOST,
    port=config.DB_PORT
)
cur = conn.cursor()

# Pygame initialization
pygame.init()
WIDTH, HEIGHT = 600, 600  # Increased height for more space
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("PhoneBook Application")
font = pygame.font.SysFont('Arial', 20)
font_large = pygame.font.SysFont('Arial', 25)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BUTTON_COLOR = (70, 130, 180)
BUTTON_HOVER_COLOR = (100, 150, 200)
BUTTON_WIDTH, BUTTON_HEIGHT = 200, 40

# Button class
class Button:
    def __init__(self, x, y, width, height, text, action=None):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.action = action

    def draw(self, screen):
        mouse_x, mouse_y = pygame.mouse.get_pos()
        color = BUTTON_HOVER_COLOR if self.rect.collidepoint(mouse_x, mouse_y) else BUTTON_COLOR
        pygame.draw.rect(screen, color, self.rect)
        text_surface = font.render(self.text, True, WHITE)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)

    def click(self):
        mouse_x, mouse_y = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_x, mouse_y) and self.action:
            self.action()

# PostgreSQL interaction functions
def load_csv():
    filename = input("Insert the path to CSV file: ")
    try:
        df = pd.read_csv(filename)
        for _, row in df.iterrows():
            cur.execute("""
                INSERT INTO contacts (username, phone_number, first_name, second_name)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (username) DO UPDATE SET
                    phone_number = EXCLUDED.phone_number,
                    first_name = EXCLUDED.first_name,
                    second_name = EXCLUDED.second_name;
            """, (row['Username'], row['Phone Number'], row['First Name'], row['Second Name']))
        conn.commit()
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Failed to load CSV: {e}")

def add_person():
    username = input("Enter Username: ")
    phone = input("Enter Phone Number: ")

    # Validate phone number length (e.g., it should be 10 digits)
    if len(phone) != 10 or not phone.isdigit():
        print("Invalid phone number. It must contain exactly 10 digits.")
        return

    first_name = input("Enter First Name: ")
    second_name = input("Enter Second Name: ")
    try:
        cur.execute("""
            INSERT INTO contacts (username, phone_number, first_name, second_name)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (username) DO NOTHING;
        """, (username, phone, first_name, second_name))
        conn.commit()
    except Exception as e:
        print(f"Error adding person: {e}")

def edit_person():
    username = input("Enter Username to edit: ")
    cur.execute("SELECT * FROM contacts WHERE username = %s", (username,))
    person = cur.fetchone()
    if person:
        new_phone = input(f"Phone Number (current: {person[1]}): ")

        # Validate phone number length (e.g., it should be 10 digits)
        if len(new_phone) != 10 or not new_phone.isdigit():
            print("Invalid phone number. It must contain exactly 10 digits.")
            return

        new_first = input(f"First Name (current: {person[2]}): ")
        new_second = input(f"Second Name (current: {person[3]}): ")
        cur.execute("""
            UPDATE contacts
            SET phone_number = %s, first_name = %s, second_name = %s
            WHERE username = %s;
        """, (new_phone, new_first, new_second, username))
        conn.commit()
    else:
        print("User not found.")

def delete_person():
    username = input("Enter Username to delete: ")
    cur.execute("DELETE FROM contacts WHERE username = %s", (username,))
    conn.commit()

# Global variables for pagination
limit = None
offset = 0

# Pagination function
def display_table():
    global limit, offset
    screen.fill(WHITE)
    y_offset = 60
    header = "Username | Phone Number | First Name | Second Name"
    screen.blit(font_large.render(header, True, BLACK), (50, y_offset))
    y_offset += 40
    pygame.draw.line(screen, BLACK, (50, y_offset), (WIDTH - 50, y_offset), 2)
    y_offset += 10

    try:
        if limit is not None:
            cur.execute("SELECT COUNT(*) FROM contacts;")
            total_records = cur.fetchone()[0]
            total_pages = (total_records // limit) + (1 if total_records % limit != 0 else 0)

            if offset // limit + 1 > total_pages:
                offset = (total_pages - 1) * limit  # Adjust offset to the last valid page

            cur.execute("SELECT * FROM get_contacts_paginated(%s, %s);", (limit, offset))
            rows = cur.fetchall()
            # Show page number
            screen.blit(font.render(f"Page: {offset // limit + 1}/{total_pages}", True, BLACK), (WIDTH - 150, HEIGHT - 70))
        else:
            cur.execute("SELECT * FROM contacts ORDER BY username;")
            rows = cur.fetchall()

        for row in rows:
            text = f"{row[0]} | {row[1]} | {row[2]} | {row[3]}"
            screen.blit(font.render(text, True, BLACK), (50, y_offset))
            y_offset += 30
    except Exception as e:
        print(f"Error displaying table: {e}")

# Function for paginated view input
def paginated_view():
    global limit, offset
    try:
        limit = int(input("Enter number of records per page (limit): "))
        offset = int(input("Enter starting offset: "))
        display_table()
    except Exception as e:
        print(f"Error with pagination input: {e}")

# Functions for page navigation
def next_page():
    global offset
    # Prevent going beyond the last page
    cur.execute("SELECT COUNT(*) FROM contacts;")
    total_records = cur.fetchone()[0]
    if offset + limit < total_records:
        offset += limit
    display_table()

def prev_page():
    global offset
    if offset >= limit:
        offset -= limit
    display_table()

# Buttons
buttons = [
    Button(50, 300, BUTTON_WIDTH, BUTTON_HEIGHT, "Upload CSV", load_csv),
    Button(270, 300, BUTTON_WIDTH, BUTTON_HEIGHT, "Add Person", add_person),
    Button(50, 350, BUTTON_WIDTH, BUTTON_HEIGHT, "Edit Person", edit_person),
    Button(270, 350, BUTTON_WIDTH, BUTTON_HEIGHT, "Delete Person", delete_person),
    Button(270, 400, BUTTON_WIDTH, BUTTON_HEIGHT, "Next Page", next_page),  # Next page button
    Button(50, 400, BUTTON_WIDTH, BUTTON_HEIGHT, "Prev Page", prev_page),   # Previous page button
    Button(150, 450, BUTTON_WIDTH, BUTTON_HEIGHT, "Paginated View", paginated_view)
]

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False
        elif event.type == MOUSEBUTTONDOWN:
            for button in buttons:
                button.click()

    display_table()
    for button in buttons:
        button.draw(screen)

    pygame.display.flip()

# Cleanup
pygame.quit()
cur.close()
conn.close()


In [ ]:
#datas.csv

Username,Phone Number,First Name,Second Name
beka,87007007070,Bekassyl,Zatybek
ers,87778887788,Ersultan,Serik
dias,800011001010,Dias,Serik

In [ ]:
#config.py

DB_USER = "postgres"
DB_PASSWORD = "*******"
DB_HOST = "localhost"
DB_PORT = "5432"